In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import pyspark.sql.types as st
from datetime import date

spark = SparkSession.builder.getOrCreate()

daily_country_confirmed_cases = "./data/Daily_Country_Wise_Confirmed_Cases.csv"

daily_df = spark.read.csv(daily_country_confirmed_cases,inferSchema=True,header=True)


In [32]:
# Change to importing this function
def date_check():
    today = date.today()
    return str(today)

from datetime import datetime, timedelta

def date_minus_weeks(x:int) -> str:
    """
    Returns a timedelta STRING
    """
    return str(date.today() - timedelta(weeks = x))

In [29]:
# Showing the change of cases over a month
daily_df.select('Country','2022-05-06', '2022-06-06').where(daily_df['2022-06-06'] >= 1).show()

+-------------+----------+----------+
|      Country|2022-05-06|2022-06-06|
+-------------+----------+----------+
|      England|         1|        73|
|     Portugal|         0|        10|
|        Spain|         0|        17|
|United States|         0|         1|
|        Italy|         0|         6|
|       Canada|         0|         1|
|     Scotland|         0|         2|
|     Slovenia|         0|         3|
|        Wales|         0|         1|
+-------------+----------+----------+



In [37]:
# Shows the change over 1 week
daily_df.select('Country', daily_df[date_minus_weeks(2)],daily_df[date_minus_weeks(1)]).where(daily_df[date_minus_weeks(1)] > 10).show()

+-------------+----------+----------+
|      Country|2022-06-22|2022-06-29|
+-------------+----------+----------+
|     Portugal|        13|        18|
|United States|        14|        45|
|      Germany|        54|        35|
|       Canada|         0|        12|
|       Brazil|         2|        16|
+-------------+----------+----------+



In [43]:
daily_df.select(
    'Country',  
    daily_df[date_minus_weeks(7)],
    daily_df[date_minus_weeks(6)],
    daily_df[date_minus_weeks(5)],
    daily_df[date_minus_weeks(4)],
    daily_df[date_minus_weeks(3)],
    daily_df[date_minus_weeks(2)],
    daily_df[date_minus_weeks(1)]) \
        .show()

+--------------------+----------+----------+----------+----------+----------+----------+----------+
|             Country|2022-05-18|2022-05-25|2022-06-01|2022-06-08|2022-06-15|2022-06-22|2022-06-29|
+--------------------+----------+----------+----------+----------+----------+----------+----------+
|             England|         2|         7|         5|         0|         0|         0|         0|
|            Portugal|        11|        10|        19|        25|        10|        13|        18|
|               Spain|         7|         8|         0|        34|         0|         0|         0|
|       United States|         1|         2|         0|         5|        12|        14|        45|
|             Germany|         0|         1|         0|        13|        45|        54|        35|
|             Belgium|         0|         2|         5|         8|        27|         0|         0|
|              Sweden|         0|         1|         0|         0|         4|         0|         0|
